# DTI 데이터 전처리

In [1]:
from IPython.core.display import display
from rdkit import Chem
from rdkit.Chem import AllChem, MolToSmiles
from Bio import SeqIO
import pandas as pd
from rdkit.Chem import PandasTools
from rdkit.Chem import Descriptors
pd.set_option('display.max_rows', 10)

RDKit WARNING: [18:28:01] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
cmp_db = PandasTools.LoadSDF('approved_structures.sdf')
cmp_db = cmp_db.drop_duplicates('DRUGBANK_ID')
cmp_db = cmp_db.reset_index(drop=True)
cmp_db.to_csv("cmp_db.csv", mode='w')
cmp_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2451 entries, 0 to 2450
Data columns (total 44 columns):
ALOGPS_LOGP                                  2302 non-null object
ALOGPS_LOGS                                  2302 non-null object
ALOGPS_SOLUBILITY                            2302 non-null object
DATABASE_ID                                  2451 non-null object
DATABASE_NAME                                2451 non-null object
DRUGBANK_ID                                  2451 non-null object
DRUG_GROUPS                                  2451 non-null object
EXACT_MASS                                   2451 non-null object
FORMULA                                      2451 non-null object
GENERIC_NAME                                 2451 non-null object
ID                                           2451 non-null object
INCHI_IDENTIFIER                             2450 non-null object
INCHI_KEY                                    2451 non-null object
INTERNATIONAL_BRANDS               

In [3]:
cmp_db["morgan_fp_r2"]=1
nMols = len(cmp_db["ROMol"])
for i in range(nMols):
    morgan_fp = AllChem.GetMorganFingerprintAsBitVect(cmp_db["ROMol"].iloc[i], radius=2, nBits=2048)
    morgan_fp=morgan_fp.ToBitString()
    cmp_db["morgan_fp_r2"].iloc[i] = '\t'.join(list(morgan_fp))
    

cmp_db.rename(columns = {'DATABASE_ID' : 'Compound_ID'}, inplace =True) 
sample_cmp = cmp_db[["Compound_ID","morgan_fp_r2"]]
#deep conv 넣으려고 열이름 걔네들꺼랑 같이 바꿔줌


sample_cmp

C:\Users\yeonuk\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Compound_ID,morgan_fp_r2
0,DB00006,0\t1\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0...
1,DB00007,0\t1\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0...
2,DB00014,0\t1\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0...
3,DB00027,0\t1\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0...
4,DB00035,0\t1\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0...
...,...,...
2446,DB15494,0\t1\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0...
2447,DB15534,0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0...
2448,DB15566,0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0...
2449,DB15576,0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0...


In [4]:
prt_db = pd.DataFrame({"Protein_ID":[],"Sequence":[]})
for records in SeqIO.parse("approved_protein.fasta","fasta"):
    prt_db = prt_db.append(pd.DataFrame(data=[[records.id, records.seq]],columns=["Protein_ID","Sequence"]))

prt_db = prt_db.drop_duplicates('Protein_ID')
prt_db = prt_db.reset_index(drop=True)
prt_db['Protein_ID'] = prt_db['Protein_ID'].str[16:]

prt_db

,Protein_ID,Sequence
0,P45059,"(M, V, K, F, N, S, S, R, K, S, G, K, S, K, K, ..."
1,P19113,"(M, M, E, P, E, E, Y, R, E, R, G, R, E, M, V, ..."
2,Q9UI32,"(M, R, S, M, K, A, L, Q, K, A, L, S, R, A, G, ..."
3,P00488,"(M, S, E, T, S, R, T, A, F, G, G, R, R, A, V, ..."
4,P35228,"(M, A, C, P, W, K, F, L, F, K, T, K, F, H, Q, ..."
...,...,...
2744,P06025,"(M, D, A, D, K, I, V, F, K, V, N, N, Q, V, V, ..."
2745,P06747,"(M, S, K, I, F, V, N, P, S, A, I, R, A, G, L, ..."
2746,P48764,"(M, W, G, L, G, A, R, G, P, D, R, G, L, L, L, ..."
2747,P11532,"(M, L, W, W, E, E, V, E, D, C, Y, E, R, E, D, ..."


In [5]:
dti_ids = pd.read_csv('drugbank_all_target_polypeptide_ids.csv/all.csv')
dti_labels = dti_ids[['UniProt ID','Drug IDs']]

In [6]:
dti_labels

,UniProt ID,Drug IDs
0,P45059,DB00303
1,P19113,DB00114; DB00117
2,Q9UI32,DB00142
3,P00488,DB01839; DB02340; DB11300; DB11311; DB11571; D...
4,P35228,DB00125; DB00155; DB01110; DB01234; DB01686; D...
...,...,...
5115,Q13003,DB00273
5116,Q16099,DB00273
5117,Q16478,DB00273
5118,P11532,DB15593


In [7]:
dti_label = pd.concat([dti_labels['UniProt ID'],pd.DataFrame(dti_labels['Drug IDs'].str.split(';').tolist())], axis=1) 
dti_label = dti_label.set_index('UniProt ID').stack().reset_index()
del dti_label['level_1']

dti_label.columns = ['Protein_ID','Compound_ID']
dti_label = dti_label.loc[:,::-1]
dti_label['Label'] = 1

dti_label = dti_label.drop_duplicates()
dti_label

,Compound_ID,Protein_ID,Label
0,DB00303,P45059,1
1,DB00114,P19113,1
2,DB00117,P19113,1
3,DB00142,Q9UI32,1
4,DB01839,P00488,1
...,...,...,...
20739,DB00273,Q13003,1
20740,DB00273,Q16099,1
20741,DB00273,Q16478,1
20742,DB15593,P11532,1


In [38]:
label_available = dti_label[dti_label.Compound_ID.isin(list(cmp_db.Compound_ID)) & dti_label.Protein_ID.isin(list(prt_db.Protein_ID))]

In [39]:
# label_available 컴파운드와 프로틴 정보가 둘다 각 db에 있어서 실제 학습에 사용될 수 있는 라벨 
label_available = dti_label[dti_label.Compound_ID.isin(list(cmp_db.Compound_ID)) & dti_label.Protein_ID.isin(list(prt_db.Protein_ID))] 

cmp_final = cmp_db[cmp_db.Compound_ID.isin(list(label_available.Compound_ID))]
prt_final = prt_db[prt_db.Protein_ID.isin(list(label_available.Protein_ID))]

cmp_final = cmp_final.reset_index(drop=True)
prt_final = prt_final.reset_index(drop=True)

In [41]:
from itertools import product

items = [list(cmp_final.Compound_ID),list(prt_final.Protein_ID)]
total_table = pd.DataFrame(product(*items),columns = ["Compound_ID", "Protein_ID"])

In [42]:
total_table = pd.merge(left=total_table, right=label_available, how='left', on=["Compound_ID","Protein_ID"], sort=False)
total_table = total_table.fillna(0)
total_table.Label = total_table.Label.astype('Int64')

total_table= total_table.reset_index(drop=True)
total_table

,Compound_ID,Protein_ID,Label
0,DB00007,P45059,0
1,DB00007,P19113,0
2,DB00007,Q9UI32,0
3,DB00007,P00488,0
4,DB00007,P37059,0
...,...,...,...
1218807,DB13997,Q9X757,0
1218808,DB13997,A0A068JFB7,0
1218809,DB13997,Q93LQ9,0
1218810,DB13997,Q8TD43,0


In [43]:
#csv로 저장
cmp_final.to_csv("drugbank_cmp_proccessed.csv", mode='w')
prt_final.to_csv("drugbank_prt.csv", mode='w')
total_table.to_csv("drugbank_label_proccessed.csv", mode='w')

In [47]:
sum(total_table.Label)

2447

In [48]:
zero_label = total_table[total_table['Label']==0] #total_table = drugbank_label.csv
sample_zero_label = zero_label.sample(frac=0.001).reset_index(drop=True) # 약 670만개에서 0.001배 하여 6735개
sample_table = pd.concat([total_table[total_table['Label']==1] , sample_zero_label]) #라벨 1인것 2447개 더해서 9182 개


In [51]:
x_data = sample_table.drop('Label',axis=1)
y_data = sample_table['Label']

In [52]:
from sklearn.model_selection import train_test_split

x_train_raw, x_test, y_train_raw, y_test = train_test_split(
    x_data, 
    y_data,
    stratify=sample_table['Label'],
    test_size=0.2, random_state=1)


x_train, x_vaild, y_train, y_vaild = train_test_split(
    x_train_raw, 
    y_train_raw,
    stratify=y_train_raw,
    test_size=0.25, random_state=1)

In [53]:
print(len(x_train),len(x_vaild), len(x_test))

5508 1837 1837


In [54]:
print(sum(y_train),sum(y_vaild),sum(y_test))

1468 489 490


In [55]:
x_train['Label'] = y_train
x_vaild['Label'] = y_vaild
x_test['Label'] = y_test

C:\Users\yeonuk\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\yeonuk\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\yeonuk\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

In [56]:
training_comp = sample_cmp[sample_cmp.Compound_ID.isin(x_train.Compound_ID.drop_duplicates())]
training_prt = prt_db[prt_db.Protein_ID.isin(x_train.Protein_ID.drop_duplicates())]

training_comp.to_csv("training_comp.csv", mode='w')
training_prt.to_csv("training_prt.csv", mode='w')
x_train.to_csv("training_label.csv", mode='w')

In [57]:
validation_comp = sample_cmp[sample_cmp.Compound_ID.isin(x_vaild.Compound_ID.drop_duplicates())]
validation_prt = prt_db[prt_db.Protein_ID.isin(x_vaild.Protein_ID.drop_duplicates())]

validation_comp.to_csv("validation_comp.csv", mode='w')
validation_prt.to_csv("validation_prt.csv", mode='w')
x_vaild.to_csv("validation_label.csv", mode='w')

In [58]:
test_comp = sample_cmp[sample_cmp.Compound_ID.isin(x_test.Compound_ID.drop_duplicates())]
test_prt = prt_db[prt_db.Protein_ID.isin(x_test.Protein_ID.drop_duplicates())]

test_comp.to_csv("test_comp.csv", mode='w')
test_prt.to_csv("test_prt.csv", mode='w')
x_test.to_csv("test_label.csv", mode='w')

In [29]:
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator

In [31]:
desclist=[]
for i in range(len(Descriptors.descList)):
    desclist.append(Descriptors.descList[i][0])
#200개

In [32]:
desc_df = pd.DataFrame(columns=desclist)
for c in range(len(cmp_db.DRUGBANK_ID)):
    desc_df.loc[c]=MolecularDescriptorCalculator(desclist).CalcDescriptors(cmp_db["ROMol"][c])


In [34]:
cmp_features = pd.concat([sample_cmp,desc_df], axis=1)
cmp_features.to_csv("cmp_features.csv", mode='w')

In [ ]:
# 연습노트

In [53]:
suppl = Chem.SDMolSupplier('approved_structures.sdf')
nMols = len(suppl)

In [ ]:
a=[]
for i in range(nMols): 
    a=print(suppl[i].GetProp('DRUGBANK_ID'))
    


In [78]:
morgan_fp.ToBitString()

'000000000100001000000000000000000000000000000000000000000000000000010100000000001000100000000000000000100000000000000000000000001000000000000000000000000000000000000000010000000000100100000001000000000000000000000000000000000000000000010100000000000000000000000000000000000000000010000000000000100000000000000000000000010010000000000000000000000000001000000000000000000000000000000000010000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000001000000000000000100000000000000000000000000000000000000010000000000000000000000000001000000100000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000001000000000000000000010010000000000000000000010000000000100000000001010000000000000000000000000000000000000000000100000000000000000001000000000000000100000000000000000000000000000001000001000000000000000000000100000000000000000000000000001000000000000000000000000000100000000000000000000000000000000000000

In [54]:
fp = []
for mol in suppl: 

    morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
    fp.append(morgan_fp)
    
fp

ArgumentError: Python argument types in
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(NoneType)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(class RDKit::ROMol mol, int radius, unsigned int nBits=2048, class boost::python::api::object invariants=[], class boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, class boost::python::api::object bitInfo=None)

In [ ]:
i=0
compiund_id = mols[i].GetProp('DRUGBANK_ID')
compiund_id

'DB00006'

In [82]:
fp = AllChem.GetMorganFingerprintAsBitVect(suppl[0], radius=2, nBits=2048)

In [83]:
bits = fp.ToBitString()
len(bits)

2048

In [ ]:
cmp_db = pd.DataFrame({"Compound_ID":[],"morgan_fp_r2":[]})


In [ ]:
type(MolToSmiles(mols[0]))

str

In [ ]:
mols[0].HasProp('DRUGBANK_ID') #이런 속성이 있는지 확인 (1 있다 0 없다.)
suppl[0].GetProp('DRUGBANK_ID') 

'DB00006'

1

In [ ]:
#print(Chem.MolToMolBlock(mols_3d[0]))

In [ ]:
Chem.GraphDescriptors(mols_3d[0])

AttributeError: module 'rdkit.Chem' has no attribute 'GraphDescriptors'

In [ ]:
import seaborn as sns
prt_length = [len(prt_db.Sequence[i]) for i in range(len(prt_db))]
hist, edges = np.histogram(prt_length, 500)

sns.distplot(prt_length, bins=500, kde=False)

In [ ]:
cmp_features = pd.read_csv("drugbank_cmp_dropUX.csv")
prt_db = pd.read_csv("drugbank_prt_dropUX.csv")
label_db = pd.read_csv("drugbank_label_dropUX.csv")
CTD_db = pd.read_csv("drugbank_CTD_dropUX.csv")

In [ ]:
label_db = label_db.drop(['Unnamed: 0'], axis=1)
prt_db = prt_db.drop(['Unnamed: 0'], axis=1)
cmp_features = cmp_features.drop(['Unnamed: 0'], axis=1)
CTD_db = CTD_db.drop(['Unnamed: 0'], axis=1)

In [ ]:
seq_length= [len(prt_db.Sequence[i]) for i in range(len(prt_db))]
prt_db = prt_db.assign(length = seq_length)
prt_db_1500 = prt_db[prt_db.length<=1500].reset_index(drop=True)
prt_db_1500.to_csv("prt_db_1500.csv", mode='w')

In [ ]:
CTD_db_1500 = CTD_db[CTD_db.Protein_ID.isin(prt_db_1500["Protein_ID"])].reset_index(drop=True)

CTD_db_1500.to_csv("CTD_db_1500.csv", mode='w')

In [ ]:
label_db_1500 = label_db[label_db.Protein_ID.isin(prt_db_1500["Protein_ID"])].reset_index(drop=True)

label_db_1500.to_csv("label_db_1500.csv", mode='w')
